In [1]:
import pandas as pd
import numpy as np
import os
import pandas_ta as ta
import pyotp
import datetime as dt
import json
import time
import requests
import datetime
import pandas_ta as ta
from datetime import timedelta

## Initialize variable values

In [2]:
#nitialize timeframe variable for using different time durations for date column . timeframe value in integer is of minutes.
# so timeframe=3, means 3 minutes intervals, timeframe=10, means 10 minutes intervals, timeframe=60, means 1 hr intervals.

timeframe = 3


base_capital = 25000
# starting_capital = 20000


# Initialize RSI values
first_upper_rsi = 70
first_lower_rsi = 60
second_upper_rsi = 40
second_lower_rsi = 30

#provide year for which the data has to be initialized. example year = 2022, year = 2023, etc
year = 2022

#set file path of data which has to be analysed
file_path = "C:/Users/ankit/Desktop/btcusd.json"

In [252]:
#read the json file from the path
df = pd.read_json(file_path)

In [253]:

df.reset_index(drop=True, inplace=True)
df['date'] = pd.to_datetime(df['time_period_start']).dt.tz_convert(None)
df['date'] = df['date'] + timedelta(seconds= 19800)
df.set_index('date',inplace=True)

#create dataframe for the timeframe interval provided during initialization of timeframe variable
df_3min = df.resample(f'{timeframe}T').agg({'rate_open': 'first', 
                                  'rate_high': 'max', 
                                  'rate_low': 'min', 
                                  'rate_close': 'last'})

df_3min.dropna(inplace=True)

In [254]:
df_3min

,rate_open,rate_high,rate_low,rate_close
date,,,,
2021-01-01 05:30:00,29039.482320,29039.482320,29010.108620,29010.108620
2021-01-01 05:33:00,29014.667185,29014.667185,28916.382320,28916.382320
2021-01-01 05:39:00,28875.919455,28893.169622,28751.785878,28751.785878
2021-01-01 05:45:00,28772.776436,28855.936264,28772.776436,28855.936264
2021-01-01 05:48:00,28876.526664,28876.526664,28823.849294,28855.063145
...,...,...,...,...
2023-06-05 01:09:00,27220.019714,27220.260260,27217.753426,27217.753426
2023-06-05 01:15:00,27224.478443,27224.478443,27215.764522,27215.764522
2023-06-05 01:18:00,27215.867925,27216.235892,27210.142007,27210.772135


In [255]:
df_3min['rsi'] = ta.rsi(df_3min['rate_close']) 

In [256]:
#last year provided in dataframe.
# last_year = df_3min.index.year[len(df_3min.index.year)-1]

# if year==last_year:
#     df_3min =  df_3min[df_3min.index.year >= last_year]
    
# elif year>=df_3min.index.year[0] and year<last_year:
#      df_3min = df_3min[(df_3min.index.year >= year) & (df_3min.index.year < year + 1)]
    
# else:
#     print("No data found in the provided year")

# Extract data from given year to last date

In [257]:
df_3min =  df_3min[df_3min.index.year >= year]

In [258]:
df_3min

,rate_open,rate_high,rate_low,rate_close,rsi
date,,,,,
2022-01-01 00:00:00,46926.085717,46964.970845,46921.926962,46964.970845,27.635519
2022-01-01 00:03:00,46942.537865,46944.379081,46907.502552,46907.502552,25.210582
2022-01-01 00:09:00,46892.884390,46892.884390,46675.587519,46737.740830,19.708941
2022-01-01 00:15:00,46700.372628,46735.595531,46700.372628,46735.595531,19.650581
2022-01-01 00:18:00,46705.502710,46707.299660,46659.162230,46659.162230,17.645767
...,...,...,...,...,...
2023-06-05 01:09:00,27220.019714,27220.260260,27217.753426,27217.753426,57.393270
2023-06-05 01:15:00,27224.478443,27224.478443,27215.764522,27215.764522,56.029878
2023-06-05 01:18:00,27215.867925,27216.235892,27210.142007,27210.772135,52.649007


In [259]:
df_3min.dropna(inplace=True)


#initiate day varaible starting from day 1
day = 0
day_list = []

#count days based on 'date+5:30:00 pm'
for t in df_3min.index.time:
    if str(t) == '05:30:00':
        day += 1
        day_list.append(day)
    else:
        day_list.append(day)

#create day coulmn based on day count done above
df_3min['day'] =  day_list 

#group by day
day_groups = df_3min.groupby("day")

C:\Users\ankit\AppData\Local\Temp\ipykernel_17300\188091626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3min.dropna(inplace=True)
C:\Users\ankit\AppData\Local\Temp\ipykernel_17300\188091626.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3min['day'] =  day_list


In [260]:
# for day, day_df in day_groups:
#     print(day_df)

In [261]:
df_3min

,rate_open,rate_high,rate_low,rate_close,rsi,day
date,,,,,,
2022-01-01 00:00:00,46926.085717,46964.970845,46921.926962,46964.970845,27.635519,0
2022-01-01 00:03:00,46942.537865,46944.379081,46907.502552,46907.502552,25.210582,0
2022-01-01 00:09:00,46892.884390,46892.884390,46675.587519,46737.740830,19.708941,0
2022-01-01 00:15:00,46700.372628,46735.595531,46700.372628,46735.595531,19.650581,0
2022-01-01 00:18:00,46705.502710,46707.299660,46659.162230,46659.162230,17.645767,0
...,...,...,...,...,...,...
2023-06-05 01:09:00,27220.019714,27220.260260,27217.753426,27217.753426,57.393270,520
2023-06-05 01:15:00,27224.478443,27224.478443,27215.764522,27215.764522,56.029878,520
2023-06-05 01:18:00,27215.867925,27216.235892,27210.142007,27210.772135,52.649007,520


In [262]:
tradebook =[]
def tradebook_entry(tradebook):
    data = {
            'entry_time':(entry_time+timedelta(minutes=timeframe)).strftime("%Y-%m-%d %H:%M:%S"),
            'entry_price': round(entry_price,2), 
            'exit_price': round(exit_price,2), 
            'exit_time': (idx+timedelta(minutes=3)).strftime("%Y-%m-%d %H:%M:%S"), 
            'side': 'buy' if is_position == 1 else 'sell', 
            'pnl': (exit_price - entry_price)*is_position}
    tradebook.append(data)


In [263]:
tradebook = []
# long_flag=0
# short_flag=0
#to check if long 
for day, day_df in day_groups:
    is_position = False
    entry_time = None
    exit_time = None
    day_df['next_open'] = day_df['rate_open'].shift(-1)  
    for idx, row in day_df.iterrows():
        candle_close = row['rate_close']
        if (idx.time()==datetime.time(15,15)) and (is_position):
            exit_price=candle_close
            exit_time = idx
            tradebook_entry(tradebook)
            print(f'Time over, trade closed at {exit_price} {idx}')
        if not is_position:
            if (first_upper_rsi < row['rsi']):
                is_position = 1
                entry_price = row['next_open']
                entry_time = idx
                print(f'Long entry at {entry_price} {idx}')
                long_flag=1
            elif(second_lower_rsi > row['rsi']):
                is_position=-1
                entry_price = row['next_open']
                print(f'Short entry at {entry_price} {idx}')
                entry_time = idx
        if is_position==1:
            if (first_lower_rsi > row['rsi']):
                exit_price = candle_close 
                tradebook_entry(tradebook)
                print(f'long close at {exit_price} {idx}')
                is_position=0
        if is_position==-1:
            if(second_upper_rsi<row['rsi']):
                exit_price = candle_close
                tradebook_entry(tradebook)
                print(f'short exit at {exit_price} {idx}')
                is_position=0


Short entry at 46942.53786525689 2022-01-01 00:00:00
short exit at 46179.9367846458 2022-01-01 01:33:00
Long entry at 47438.614088817616 2022-01-01 10:48:00
long close at 47185.767333922464 2022-01-01 11:24:00
Long entry at 47384.99069343752 2022-01-01 21:03:00
long close at 47224.74773722022 2022-01-01 21:15:00
Long entry at 47756.48766465783 2022-01-01 23:03:00
long close at 47656.95462834785 2022-01-01 23:54:00
Short entry at 46955.88053743938 2022-01-02 10:15:00
short exit at 47037.88697973092 2022-01-02 11:00:00
Short entry at 47102.9330392859 2022-01-02 17:33:00
short exit at 47182.39174898556 2022-01-02 17:48:00
Long entry at 47595.1694057282 2022-01-02 22:00:00
long close at 47549.84822828292 2022-01-02 22:39:00
Short entry at 46955.1834911477 2022-01-02 23:54:00
short exit at 47008.45898884558 2022-01-03 00:24:00
Long entry at 47304.850160070935 2022-01-03 04:03:00
long close at 47269.71827578188 2022-01-03 04:24:00
Long entry at 47487.12041533084 2022-01-03 17:39:00
long clos

Short entry at 36338.17304601155 2022-01-27 07:00:00
short exit at 36012.61600552706 2022-01-27 08:15:00
Short entry at 35613.69716526441 2022-01-27 09:09:00
short exit at 35804.88173988803 2022-01-27 09:33:00
Long entry at 36673.90751649364 2022-01-28 04:09:00
Long entry at 37385.78135410709 2022-01-28 05:33:00
long close at 37148.62278817591 2022-01-28 06:15:00
Long entry at 37434.342795383025 2022-01-28 08:39:00
long close at 37306.54187334508 2022-01-28 09:45:00
Short entry at 36972.35112000346 2022-01-28 12:30:00
short exit at 37104.32962357656 2022-01-28 12:45:00
Short entry at 36581.07038535636 2022-01-28 13:48:00
short exit at 36666.97474500426 2022-01-28 14:39:00
Long entry at 37645.79702021885 2022-01-29 02:09:00
long close at 37842.76941133752 2022-01-29 03:54:00
Long entry at 37889.422173166306 2022-01-29 09:54:00
long close at 37825.93370381083 2022-01-29 10:00:00
Long entry at 38081.01281120871 2022-01-29 19:03:00
long close at 38072.891828554246 2022-01-29 20:00:00
Short

Short entry at 37457.659818203196 2022-02-22 03:18:00
short exit at 37431.67563899611 2022-02-22 04:09:00
Short entry at 36552.399415807144 2022-02-22 08:33:00
short exit at 36649.38023098383 2022-02-22 09:30:00
Long entry at 37218.651611425936 2022-02-22 11:15:00
long close at 36772.48685255661 2022-02-22 11:24:00
Long entry at 37476.26119082617 2022-02-22 15:54:00
long close at 37584.28626504809 2022-02-22 16:48:00
Long entry at 38369.11767965012 2022-02-23 05:09:00
Short entry at 37721.62719986547 2022-02-23 08:00:00
short exit at 37799.693075808325 2022-02-23 08:54:00
Long entry at 38765.352944323546 2022-02-23 13:54:00
Time over, trade closed at 38802.89125446568 2022-02-23 15:15:00
long close at 38807.30819572242 2022-02-23 15:39:00
Long entry at 39144.71366669771 2022-02-23 18:45:00
long close at 39057.02592805213 2022-02-23 18:48:00
Short entry at 38637.0978078433 2022-02-23 20:30:00
short exit at 38513.67758217159 2022-02-23 21:54:00
Short entry at 38138.96821524861 2022-02-23

Short entry at 38776.08718357897 2022-03-14 01:30:00
short exit at 38810.61829645539 2022-03-14 02:39:00
Short entry at 37911.33079686288 2022-03-14 04:15:00
Short entry at 37800.63082564562 2022-03-14 05:30:00
short exit at 37833.60877657098 2022-03-14 05:54:00
Long entry at 38678.97621316122 2022-03-14 09:39:00
long close at 38469.033786739164 2022-03-14 10:30:00
Long entry at 39055.03679556079 2022-03-14 13:00:00
long close at 38977.64935929705 2022-03-14 13:54:00
Short entry at 38769.07793361127 2022-03-14 18:18:00
short exit at 38831.226135209516 2022-03-14 18:24:00
Long entry at 39141.27856979385 2022-03-15 04:09:00
Short entry at 38987.713943792594 2022-03-15 06:45:00
short exit at 39095.73470092344 2022-03-15 08:03:00
Short entry at 38665.800733722426 2022-03-15 09:09:00
short exit at 38827.552630803555 2022-03-15 09:45:00
Short entry at 38259.017027364156 2022-03-15 12:39:00
short exit at 38457.6683769123 2022-03-15 13:18:00
Long entry at 38893.594502139385 2022-03-15 18:00:00

Long entry at 46687.6059668712 2022-04-03 20:30:00
long close at 46528.40398642054 2022-04-03 20:54:00
Long entry at 46637.27374339109 2022-04-04 00:18:00
long close at 46509.29812694519 2022-04-04 00:30:00
Long entry at 46711.53281482654 2022-04-04 02:54:00
long close at 46595.75728032117 2022-04-04 03:00:00
Long entry at 46770.829109479026 2022-04-04 03:30:00
long close at 46931.61388791402 2022-04-04 04:15:00
Short entry at 45999.563231967215 2022-04-04 05:54:00
short exit at 45926.053760780065 2022-04-04 07:45:00
Short entry at 45692.95005034649 2022-04-04 20:33:00
short exit at 45768.98647230099 2022-04-04 21:39:00
Short entry at 45343.32493943499 2022-04-04 23:09:00
short exit at 45376.89001971241 2022-04-05 00:03:00
Long entry at 46179.21858811013 2022-04-05 01:48:00
long close at 46436.40610977798 2022-04-05 03:39:00
Long entry at 46778.619084702936 2022-04-05 04:15:00
long close at 46704.99125687831 2022-04-05 05:03:00
Long entry at 46999.856608206705 2022-04-05 17:48:00
long 

...

short exit at 21674.807221067193 2023-03-09 14:03:00
Long entry at 21811.689741623093 2023-03-09 19:45:00
long close at 21722.01898379712 2023-03-09 20:00:00
Short entry at 21499.95293769961 2023-03-09 22:15:00
short exit at 20338.988599470085 2023-03-10 03:33:00
Short entry at 20113.07232758723 2023-03-10 05:48:00
short exit at 20048.82971319161 2023-03-10 07:00:00
Short entry at 19944.82052331937 2023-03-10 10:00:00
short exit at 19954.80484462988 2023-03-10 10:48:00
Short entry at 19867.794826504676 2023-03-10 12:24:00
short exit at 19934.177774984142 2023-03-10 12:48:00
Short entry at 19697.23630170406 2023-03-10 16:09:00
short exit at 19685.31030008656 2023-03-10 17:15:00
Long entry at 20076.161767717676 2023-03-10 19:00:00
long close at 19979.928455054123 2023-03-10 19:33:00
Long entry at 20274.587173000204 2023-03-10 22:00:00
long close at 20116.647554881994 2023-03-10 22:09:00
Long entry at 20163.852248347986 2023-03-11 03:30:00
long close at 20120.838173044445 2023-03-11 04:15

Short entry at 27554.168170838922 2023-03-24 17:15:00
short exit at 28041.10331352951 2023-03-24 17:33:00
Short entry at 27708.13816570528 2023-03-24 22:54:00
short exit at 27711.783509448243 2023-03-24 23:39:00
Short entry at 27436.697148332147 2023-03-25 02:39:00
short exit at 27418.910027377573 2023-03-25 03:30:00
Short entry at 27471.746569124884 2023-03-25 11:09:00
short exit at 27498.668051901233 2023-03-25 11:33:00
Long entry at 27660.417085740086 2023-03-25 19:33:00
long close at 27581.722546023397 2023-03-25 20:09:00
Long entry at 27729.170103593475 2023-03-25 21:15:00
long close at 27681.62384278908 2023-03-25 21:33:00
Short entry at 27328.894389915466 2023-03-26 00:30:00
short exit at 27306.847184975395 2023-03-26 01:33:00
Long entry at 27613.470392276984 2023-03-26 06:15:00
long close at 27547.79659551474 2023-03-26 06:30:00
Long entry at 27719.356491196104 2023-03-26 14:03:00
long close at 27679.792862814076 2023-03-26 15:00:00
Long entry at 27890.994806230527 2023-03-26 1

Long entry at 30453.50868763302 2023-04-12 18:00:00
long close at 30212.273197863055 2023-04-12 18:30:00
Short entry at 29743.483379840563 2023-04-12 20:39:00
short exit at 29908.678388995497 2023-04-12 20:48:00
Long entry at 30163.961001646945 2023-04-13 08:09:00
long close at 30101.091943601474 2023-04-13 08:45:00
Long entry at 30327.092093397645 2023-04-13 14:39:00
long close at 30233.309023918908 2023-04-13 15:00:00
Long entry at 30462.993402872435 2023-04-13 20:03:00
long close at 30425.21850160181 2023-04-13 20:33:00
Short entry at 30294.783472555453 2023-04-14 01:33:00
short exit at 30326.205715326276 2023-04-14 02:00:00
Long entry at 30801.015272707773 2023-04-14 06:03:00
long close at 30771.85869479063 2023-04-14 07:48:00
Long entry at 30827.680634282566 2023-04-14 11:00:00
long close at 30802.76660026679 2023-04-14 11:03:00
Long entry at 30942.616668160405 2023-04-14 12:18:00
long close at 30923.907790248 2023-04-14 12:33:00
Short entry at 30641.4694941999 2023-04-14 18:24:00

Long entry at 28718.438776257637 2023-05-04 02:45:00
long close at 29049.913005679547 2023-05-04 04:30:00
Long entry at 29227.45020496579 2023-05-04 16:00:00
long close at 29207.48324109805 2023-05-04 16:30:00
Short entry at 28945.113588770295 2023-05-04 18:48:00
short exit at 28838.139570586896 2023-05-04 19:48:00
Long entry at 28929.184431182006 2023-05-05 06:09:00
long close at 28915.62999171551 2023-05-05 06:33:00
Long entry at 29172.595414329542 2023-05-05 07:00:00
long close at 29250.806556226 2023-05-05 08:24:00
Short entry at 28939.87132896604 2023-05-05 18:00:00
short exit at 29071.691227870615 2023-05-05 18:15:00
Long entry at 29384.20689018257 2023-05-05 20:03:00
long close at 29286.08031305198 2023-05-05 20:24:00
Long entry at 29570.19632463611 2023-05-05 21:30:00
long close at 29495.81448088823 2023-05-05 22:03:00
Long entry at 29810.072799728183 2023-05-06 06:03:00
long close at 29697.52033986658 2023-05-06 06:24:00
Short entry at 29410.326890746575 2023-05-06 09:00:00
sh

Long entry at 26639.388623974257 2023-05-26 19:33:00
long close at 26762.028502192286 2023-05-26 21:18:00
Short entry at 26624.366233470988 2023-05-26 23:33:00
short exit at 26706.83665293246 2023-05-26 23:54:00
Short entry at 26696.31906787417 2023-05-27 04:24:00
short exit at 26736.05265546062 2023-05-27 04:45:00
Short entry at 26678.942671410437 2023-05-27 05:48:00
short exit at 26677.175482001898 2023-05-27 06:30:00
Long entry at 26779.881510925505 2023-05-27 09:09:00
long close at 26747.694926233286 2023-05-27 09:45:00
Long entry at 26789.988679319347 2023-05-27 10:15:00
long close at 26768.292240000785 2023-05-27 10:24:00
Short entry at 26700.322247190652 2023-05-27 13:39:00
short exit at 26713.798788485932 2023-05-27 13:54:00
Long entry at 26775.21437699975 2023-05-27 18:54:00
long close at 26744.74577279937 2023-05-27 19:09:00
Short entry at 26638.51561883039 2023-05-27 19:24:00
short exit at 26700.25337839936 2023-05-27 19:48:00
Long entry at 26740.084773056667 2023-05-27 23:0

In [264]:
df_3min

,rate_open,rate_high,rate_low,rate_close,rsi,day
date,,,,,,
2022-01-01 00:00:00,46926.085717,46964.970845,46921.926962,46964.970845,27.635519,0
2022-01-01 00:03:00,46942.537865,46944.379081,46907.502552,46907.502552,25.210582,0
2022-01-01 00:09:00,46892.884390,46892.884390,46675.587519,46737.740830,19.708941,0
2022-01-01 00:15:00,46700.372628,46735.595531,46700.372628,46735.595531,19.650581,0
2022-01-01 00:18:00,46705.502710,46707.299660,46659.162230,46659.162230,17.645767,0
...,...,...,...,...,...,...
2023-06-05 01:09:00,27220.019714,27220.260260,27217.753426,27217.753426,57.393270,520
2023-06-05 01:15:00,27224.478443,27224.478443,27215.764522,27215.764522,56.029878,520
2023-06-05 01:18:00,27215.867925,27216.235892,27210.142007,27210.772135,52.649007,520


In [265]:
tradebook_df = pd.DataFrame(tradebook)
tradebook_df

,entry_time,entry_price,exit_price,exit_time,side,pnl
0,2022-01-01 00:03:00,46942.54,46179.94,2022-01-01 01:36:00,sell,762.601081
1,2022-01-01 10:51:00,47438.61,47185.77,2022-01-01 11:27:00,buy,-252.846755
2,2022-01-01 21:06:00,47384.99,47224.75,2022-01-01 21:18:00,buy,-160.242956
3,2022-01-01 23:06:00,47756.49,47656.95,2022-01-01 23:57:00,buy,-99.533036
4,2022-01-02 10:18:00,46955.88,47037.89,2022-01-02 11:03:00,sell,-82.006442
...,...,...,...,...,...,...
2330,2023-06-04 06:33:00,26972.68,27039.15,2023-06-04 06:42:00,sell,-66.468033
2331,2023-06-04 12:12:00,27120.61,27129.81,2023-06-04 13:18:00,buy,9.204130
2332,2023-06-04 14:42:00,27201.74,27233.83,2023-06-04 15:18:00,buy,32.090426
2333,2023-06-04 14:42:00,27201.74,27214.69,2023-06-04 16:12:00,buy,12.953177


In [266]:
# tradebook_df['pnl']=tradebook_df['pnl']*25
tradebook_df.pnl.cumsum()

0       762.601081
1       509.754326
2       349.511369
3       249.978333
4       167.971891
           ...    
2330    586.556901
2331    595.761032
2332    627.851458
2333    640.804635
2334    594.881043
Name: pnl, Length: 2335, dtype: float64

In [267]:
tradebook_df

,entry_time,entry_price,exit_price,exit_time,side,pnl
0,2022-01-01 00:03:00,46942.54,46179.94,2022-01-01 01:36:00,sell,762.601081
1,2022-01-01 10:51:00,47438.61,47185.77,2022-01-01 11:27:00,buy,-252.846755
2,2022-01-01 21:06:00,47384.99,47224.75,2022-01-01 21:18:00,buy,-160.242956
3,2022-01-01 23:06:00,47756.49,47656.95,2022-01-01 23:57:00,buy,-99.533036
4,2022-01-02 10:18:00,46955.88,47037.89,2022-01-02 11:03:00,sell,-82.006442
...,...,...,...,...,...,...
2330,2023-06-04 06:33:00,26972.68,27039.15,2023-06-04 06:42:00,sell,-66.468033
2331,2023-06-04 12:12:00,27120.61,27129.81,2023-06-04 13:18:00,buy,9.204130
2332,2023-06-04 14:42:00,27201.74,27233.83,2023-06-04 15:18:00,buy,32.090426
2333,2023-06-04 14:42:00,27201.74,27214.69,2023-06-04 16:12:00,buy,12.953177


In [268]:
tradebook_df['entry_time'] = pd.to_datetime(tradebook_df['entry_time'])
tradebook_df['exit_time'] = pd.to_datetime(tradebook_df['exit_time'])
tradebook_df['date'] = tradebook_df['entry_time'].dt.date
daily_pnl = tradebook_df.groupby('date')['pnl'].sum()
daily_pnl = daily_pnl.rename('ret')


daily_pnl

date
2022-01-01    249.978333
2022-01-02   -260.061827
2022-01-03   -346.043192
2022-01-04   -168.776965
2022-01-05    183.895277
                 ...    
2023-05-31    -94.516869
2023-06-01   -228.602944
2023-06-02   -132.381166
2023-06-03    -90.761650
2023-06-04    -84.296843
Name: ret, Length: 514, dtype: float64

In [269]:
daily_pnl_df = daily_pnl.reset_index()
daily_pnl_df = daily_pnl_df.rename(columns={'pnl': 'ret'})
daily_pnl_df = daily_pnl.to_frame()
daily_pnl_df['ret']

date
2022-01-01    249.978333
2022-01-02   -260.061827
2022-01-03   -346.043192
2022-01-04   -168.776965
2022-01-05    183.895277
                 ...    
2023-05-31    -94.516869
2023-06-01   -228.602944
2023-06-02   -132.381166
2023-06-03    -90.761650
2023-06-04    -84.296843
Name: ret, Length: 514, dtype: float64

In [270]:
pnl_df = daily_pnl_df.copy()
pnl_df = pnl_df.rename(columns={'ret': 'pnl'})
pnl_df['pnl_pct'] =100*(pnl_df['pnl']/base_capital)
pnl_df["pnl_pct_cumulative"] = pnl_df["pnl_pct"].cumsum()
pnl_df

,pnl,pnl_pct,pnl_pct_cumulative
date,,,
2022-01-01,249.978333,0.999913,0.999913
2022-01-02,-260.061827,-1.040247,-0.040334
2022-01-03,-346.043192,-1.384173,-1.424507
2022-01-04,-168.776965,-0.675108,-2.099615
2022-01-05,183.895277,0.735581,-1.364033
...,...,...,...
2023-05-31,-94.516869,-0.378067,4.523695
2023-06-01,-228.602944,-0.914412,3.609283
2023-06-02,-132.381166,-0.529525,3.079758


In [271]:
pnl_df.reset_index(inplace=True)
pnl_df.at[0, "pnl"] = base_capital + pnl_df.at[0, "pnl"]
pnl_df

,date,pnl,pnl_pct,pnl_pct_cumulative
0,2022-01-01,25249.978333,0.999913,0.999913
1,2022-01-02,-260.061827,-1.040247,-0.040334
2,2022-01-03,-346.043192,-1.384173,-1.424507
3,2022-01-04,-168.776965,-0.675108,-2.099615
4,2022-01-05,183.895277,0.735581,-1.364033
...,...,...,...,...
509,2023-05-31,-94.516869,-0.378067,4.523695
510,2023-06-01,-228.602944,-0.914412,3.609283
511,2023-06-02,-132.381166,-0.529525,3.079758
512,2023-06-03,-90.761650,-0.363047,2.716712


In [272]:
a=pnl_df['pnl']

In [273]:
import numpy as np
import matplotlib.pyplot as plt

def plot_equity_curve(pnl_list, starting_capital):
    equity_curve = np.cumsum(pnl_list)
    plt.plot(equity_curve)
    plt.xlabel('Day')
    plt.ylabel('Equity')
    plt.title('Equity Curve')
    plt.savefig('equity_curve_'+ str(year) +'.png')
    plt.close()

def plot_drawdown_curve(pnl_list, starting_capital):
    equity_curve = np.cumsum(pnl_list)
    drawdown = np.maximum.accumulate(equity_curve) - equity_curve
    plt.plot(drawdown)
    plt.xlabel('Day')
    plt.ylabel('Drawdown')
    plt.title('Drawdown Curve')
    plt.savefig('drawdown_curve_' + str(year) +'.png')
    plt.close()

# Example usage:
# a = [100, 200, -50, 150, -100, 50, 75, -25, -75, 100]
starting_capital = 200000

plot_equity_curve(a, starting_capital)
plot_drawdown_curve(a, starting_capital)


In [274]:
pnl_df

,date,pnl,pnl_pct,pnl_pct_cumulative
0,2022-01-01,25249.978333,0.999913,0.999913
1,2022-01-02,-260.061827,-1.040247,-0.040334
2,2022-01-03,-346.043192,-1.384173,-1.424507
3,2022-01-04,-168.776965,-0.675108,-2.099615
4,2022-01-05,183.895277,0.735581,-1.364033
...,...,...,...,...
509,2023-05-31,-94.516869,-0.378067,4.523695
510,2023-06-01,-228.602944,-0.914412,3.609283
511,2023-06-02,-132.381166,-0.529525,3.079758
512,2023-06-03,-90.761650,-0.363047,2.716712


In [275]:
import numpy as np
import pandas as pd

def calculate_metrics(pnl_list, starting_capital):
    equity_curve = np.cumsum(pnl_list)
#     print(equity_curve)
    drawdown = np.maximum.accumulate(equity_curve) - equity_curve
    max_drawdown = np.max(drawdown)
    
    max_return = np.max(pnl_list)
    max_loss = np.min(pnl_list)
    
    #current_capital = equity_curve[-1]
    current_capital = equity_curve[len(equity_curve)-1]
    max_return_percentage = (max_return / current_capital) * 100
    max_loss_percentage = (max_loss / current_capital) * 100
    
    calmar_ratio = current_capital / max_drawdown if max_drawdown != 0 else 0
    
    profitable_days = len([pnl for pnl in pnl_list if pnl > 0])
    loss_days = len([pnl for pnl in pnl_list if pnl < 0])
    
    pnl_std = np.std(pnl_list)
    sharpe_ratio = np.mean(pnl_list) / pnl_std if pnl_std != 0 else 0
    
    negative_pnls = [pnl for pnl in pnl_list if pnl < 0]
    sortino_ratio = (np.mean(negative_pnls) / np.std(negative_pnls)) * np.sqrt(len(negative_pnls)) if len(negative_pnls) != 0 else 0
    
    cumulative_return = (current_capital - starting_capital) / starting_capital * 100
    
#     pnl_df['date'] = pd.to_datetime(pnl_df.date, format='%Y-%m-%d')
#     last_month = str((pnl_df['date'].dt.month)[len(pnl_df['date'])-1])
#     last_date = str((pnl_df['date'].dt.day)[len(pnl_df['date'])-1])

    pnl_df = pd.DataFrame({'date': pd.date_range(start= str(year) + '-01-01', periods=len(pnl_list), freq='B'),
                               'pnl': pnl_list})
        
    pnl_df['date'] = pd.to_datetime(pnl_df['date'])
    pnl_df['month'] = pnl_df['date'].dt.to_period('M')
    returns_per_month = pnl_df.groupby('month')['pnl'].sum() / starting_capital * 100
    returns_per_month
    return (max_drawdown, max_return, max_loss, max_return_percentage, max_loss_percentage, calmar_ratio,
            profitable_days, loss_days, sharpe_ratio, sortino_ratio, cumulative_return, returns_per_month)


In [276]:
pnl_df

,date,pnl,pnl_pct,pnl_pct_cumulative
0,2022-01-01,25249.978333,0.999913,0.999913
1,2022-01-02,-260.061827,-1.040247,-0.040334
2,2022-01-03,-346.043192,-1.384173,-1.424507
3,2022-01-04,-168.776965,-0.675108,-2.099615
4,2022-01-05,183.895277,0.735581,-1.364033
...,...,...,...,...
509,2023-05-31,-94.516869,-0.378067,4.523695
510,2023-06-01,-228.602944,-0.914412,3.609283
511,2023-06-02,-132.381166,-0.529525,3.079758
512,2023-06-03,-90.761650,-0.363047,2.716712


In [277]:
a=pnl_df['pnl']

In [278]:
pnl_list = a

In [279]:



(max_drawdown, max_return, max_loss, max_return_percentage, max_loss_percentage, calmar_ratio,
 profitable_days, loss_days, sharpe_ratio, sortino_ratio,
 cumulative_return, returns_per_month) = calculate_metrics(pnl_list, starting_capital)

print("Max Drawdown:", max_drawdown)
print("Max Return in a Day:", max_return)
print("Max Loss in a Day:", max_loss)
print("% Max Return in a Day:", max_return_percentage)
print("% Max Loss in a Day:", max_loss_percentage)
print("Calmar Ratio:", calmar_ratio)
print("No. of Profitable Days:", profitable_days)
print("No. of Loss Days:", loss_days)
print("Sharpe Ratio:", sharpe_ratio)
print("Sortino Ratio:", sortino_ratio)
print("Cumulative Return%:", cumulative_return)
print("Returns per Month:")
print(returns_per_month)

top_1_percent = np.percentile(a, 99)
bottom_1_percent = np.percentile(a, 1)

top_10_percent = np.percentile(a, 90)
bottom_10_percent = np.percentile(a, 10)

print("Top 1%:", top_1_percent)
print("Bottom 1%:", bottom_1_percent)


print("Top 10%:", top_10_percent)
print("Bottom 10%:", bottom_10_percent)


Max Drawdown: 6956.347562803123
Max Return in a Day: 25249.978333188665
Max Loss in a Day: -1733.7143268534273
% Max Return in a Day: 98.6524543341044
% Max Loss in a Day: -6.773676048405992
Calmar Ratio: 3.6793562731454683
No. of Profitable Days: 187
No. of Loss Days: 327
Sharpe Ratio: 0.04157529406301188
Sortino Ratio: -18.679928653778568
Cumulative Return%: -87.20255947831006
Returns per Month:
month
2022-01    12.734003
2022-02    -1.811772
2022-03     1.342767
2022-04     0.220320
2022-05    -1.081751
2022-06     0.974634
2022-07    -0.133359
2022-08     1.234007
2022-09     0.770676
2022-10     0.269120
2022-11     0.340172
2022-12     0.104982
2023-01    -0.138000
2023-02    -0.399837
2023-03    -0.488904
2023-04     0.230434
2023-05    -0.439577
2023-06     0.289370
2023-07    -1.323362
2023-08     0.955195
2023-09    -0.804499
2023-10     0.520302
2023-11    -0.405769
2023-12    -0.161713
Freq: M, Name: pnl, dtype: float64
Top 1%: 1680.5423033046116
Bottom 1%: -898.83944069515